In [282]:
##importing the required librarires

import pandas as pd
import numpy as np
import datetime as dt
import time
from alpha_vantage.timeseries import TimeSeries


In [283]:
class ScriptData(object):
    
    #initializing constructor 
    def __init__(self):
        self.main_var = pd.DataFrame()
        self.data = {}
    ## get method for cllecting json data 
    def __getJson__(self):
        return self.data
    ## set method for setting json data 
    def __setJson__(self,newkey):
            self.data =newkey
    ## set method for overloading dataframes for diffrent scripts
    def __setDataFrame__(self,newkey):
        self.main_var = newkey
    
    ## get method for getting dataframes for diffrent scripts
    def __getDataFrame__(self):
        return self.main_var
        
    ##this function will fetch data in json format
    def fetch_intraday_data(self,script):
        ts = TimeSeries(key='PBUU95WNPEBUZYZI')
# Get json object with the intraday data and another with  the call's metadata
        data,meta = ts.get_intraday(script,"60min")
        self.__setJson__(data)
        #print(type(self.__getJson__()))
    
    ##this function will convert json data into pandas data frame
    def convert_intraday_data(self,script):
        df = pd.DataFrame.from_dict(self.data)
    
        df= df.transpose()
        df.reset_index(level = 0, inplace=True)
        df.columns = ["timestamp","open","high","low","close","volume"]
        df.sort_values(by='timestamp', inplace= True)
        df= df.reset_index()

        self.__setDataFrame__(df)## we are setting dataframeto main_varvariableofthe class which willconsist of the main data frame

In [284]:
script_data= ScriptData()## object of class ScriptData
script_data.fetch_intraday_data("NVDA")
script_data.convert_intraday_data("NVDA")
script_data.main_var
#script_data.plot(script_data.main_var)

,index,timestamp,open,high,low,close,volume
0,99,2022-12-21 17:00:00,165.0100,165.4300,164.5000,164.9300,440864
1,98,2022-12-21 18:00:00,164.9300,165.0000,164.9300,165.0000,5836
2,97,2022-12-21 19:00:00,164.9700,165.0900,164.9300,165.0900,8974
3,96,2022-12-21 20:00:00,165.0000,165.0000,164.4301,164.5600,21354
4,95,2022-12-22 05:00:00,165.2400,165.2400,164.4000,164.4000,3145
...,...,...,...,...,...,...,...
95,4,2022-12-30 16:00:00,144.0700,146.2899,144.0334,146.1450,5666399
96,3,2022-12-30 17:00:00,146.1400,146.7500,145.9000,146.2375,596618
97,2,2022-12-30 18:00:00,146.3000,146.4199,146.2300,146.3300,11863
98,1,2022-12-30 19:00:00,146.3900,146.4000,146.3200,146.3600,9736


In [285]:
###calculating moving average
### function to calculate moving average on close data point
def indicator1(df,timeperiod):
    temp= pd.DataFrame()
    temp["close"] = df["close"]
    temp["MA"] = temp["close"].rolling(timeperiod).mean()
    indicator_df = pd.DataFrame()
    indicator_df["timestamp"] = df["timestamp"]
    indicator_df["indicator"] = temp["MA"]
    return indicator_df

indicator1(script_data.main_var,5)

,timestamp,indicator
0,2022-12-21 17:00:00,NaN
1,2022-12-21 18:00:00,NaN
2,2022-12-21 19:00:00,NaN
3,2022-12-21 20:00:00,NaN
4,2022-12-22 05:00:00,164.7960
...,...,...
95,2022-12-30 16:00:00,144.7953
96,2022-12-30 17:00:00,145.1124
97,2022-12-30 18:00:00,145.3865
98,2022-12-30 19:00:00,145.8245


In [286]:
### defining our strategy 

In [287]:
class Strategy:
    def __init__(self,script):
        self.script = script
    ##fetching the required data
    def get_script_data(self):
        script_data.fetch_intraday_data(self.script)
        script_data.convert_intraday_data(self.script)
    
    ##function to geerate signals
    def get_signal(self):
        signals = pd.DataFrame()
        signals["timestamp"] = script_data.main_var["timestamp"]
        signals["signal"] = None
        close_data = script_data.main_var["close"]
        indicator_data = indicator1(script_data.main_var,5)["indicator"]
        temp = pd.DataFrame()###temproray dtaframe
        temp["prev_indicator"] = indicator_data.shift(1)
        temp["indicator"] = indicator_data
        temp["close"] = close_data
        temp = temp.astype(float)

            
        prev_indicator = indicator_data.shift(1)##prev indicator value in column
        prev_close = temp["close"].shift(1)##prev closevalue in column
        
        
        
        ##defining cut_upwards , cut_downward, 
        ## if the current indicator value is greater than close and previous indicator value is less then previous the it is a "BULLISH" crossover
        ##if the current indicator value is less than close and previous indicator value is greater then previous the it is a "BEARISH" crossover

        upward_crossing  = ((temp['indicator'] > temp['close']) & (prev_indicator < prev_close))## returns true if cndition is matced
        downward_crossing = ((temp['indicator'] < temp['close']) & (prev_indicator > prev_close))      
        

        temp["upward_crossing"] = upward_crossing
        temp["downward_crossing"]= downward_crossing
        #print(signals) 
        ## here we are assigning BUY and sell value if the value of upward_crossing& downward_Crossing is TRUE
        for i in range(len(temp["upward_crossing"])):
            if temp["upward_crossing"][i] == True:
                signals["signal"][i] = "BUY"
        for i in range(len(temp["downward_crossing"])):
            if temp["downward_crossing"][i] == True:
                signals["signal"][i] = "SELL"
            #if there is no buy and sell seignal we assign that column with "NO SISGNAL"
            elif signals["signal"][i] == None:
                signals["signal"][i] = "NO SIGNAL"
                
        signal_lst = ["BUY","SELL"]
        #print(temp)
        ##returning the dataframe with only buy and sell value 
        return signals[signals.signal.isin(signal_lst)]
        
        
strat = Strategy("NVDA")
strat.get_script_data()
strat.get_signal()

,timestamp,signal
14,2022-12-22 15:00:00,SELL
19,2022-12-22 20:00:00,BUY
21,2022-12-23 06:00:00,SELL
24,2022-12-23 09:00:00,BUY
28,2022-12-23 13:00:00,SELL
34,2022-12-23 19:00:00,BUY
36,2022-12-27 05:00:00,SELL
39,2022-12-27 08:00:00,BUY
53,2022-12-28 06:00:00,SELL
56,2022-12-28 09:00:00,BUY


In [288]:
#!pip install pyalgotrading

In [292]:
# from pyalgotrading.utils.candlesticks.heikinashi import HeikinAshi
# df = script_data.main_var
# df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
# df = df.astype({col: float for col in df.columns[1:]})



In [ ]:
HeikinAshi(df)


In [291]:
from pyalgotrading.utils import func
from pyalgotrading.constants import PlotType

indicator_data = indicator1(script_data.main_var,5)["indicator"]

func.plot_candlestick_chart(df,PlotType.HEIKINASHI)

In [ ]:
#!pip freeze > requirements.txt